<a href="https://colab.research.google.com/github/mattocanas/Protein-GPT2-Infilling/blob/main/GPT_Infilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install accelerate -U

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 15.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t36_3B_UR50D")

In [ ]:
import pandas as pd
import numpy as np

def infill_sequences(df, column):
    def infill_single_sequence(sequence):
        # choose a random integer between 10-20
        span_length = np.random.randint(10, 21)

        # make sure the sequence is long enough
        if len(sequence) < span_length:
            return sequence

        # choose a random start index for the span
        start_idx = np.random.randint(0, len(sequence) - span_length + 1)

        # extract and remove the span from the sequence
        span = sequence[start_idx : start_idx + span_length]
        sequence = sequence[:start_idx] + sequence[start_idx + span_length:]

        # append the span to the end of the sequence
        sequence += span

        return sequence

    df[column] = df[column].apply(infill_single_sequence)

    return df

# usage
df = pd.DataFrame({"sequences": ["EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPRVGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCARGRPLSGTGGHHYFDYWGQGTLVTVSS",'EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPRVGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCARGRPLSGTGGHHYFDYWGQGTLVTVSS','EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPRVGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCARGRPLSGTGGHHYFDYWGQGTLVTVSS','EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPRVGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTAVYYCARGRPLSGTGGHHYFDYWGQGTLVTVSS']})
df = infill_sequences(df, "sequences")
print(df.iloc[0,0])


EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPRVGSTSYAQKFQGRVTMTRDTSTSTVYMELSSLRSEDTTGGHHYFDYWGQGTLVTVSSAVYYCARGRPLSG


In [ ]:
from sklearn.model_selection import train_test_split

train_sequences, test_sequences = train_test_split(df['sequences'],  test_size=0.25, shuffle=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
train_tokenized = tokenizer(list(train_sequences))
test_tokenized = tokenizer(list(test_sequences))

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 3
})

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [ ]:
training_args = TrainingArguments(
    output_dir="/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,5.562587
2,No log,5.449945
3,No log,5.398583


TrainOutput(global_step=3, training_loss=5.6736100514729815, metrics={'train_runtime': 12.0708, 'train_samples_per_second': 0.746, 'train_steps_per_second': 0.249, 'total_flos': 172241510400.0, 'train_loss': 5.6736100514729815, 'epoch': 3.0})

In [ ]:
prompt = 'EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPR'
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=10, do_sample=True, top_k=2, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPRGQGLEWMGIINPRGQ']

In [ ]:
def remove_span(sequence, start, end):
    # remove span from start to end
    removed_span = sequence[start:end+1]
    new_sequence = sequence[:start] + sequence[end+1:]
    print("Removed span:", removed_span)
    return new_sequence

prompt = 'EVQLVQSGAEVKKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPR'
start, end = 10, 20  # specify start and end positions

# remove span
new_prompt = remove_span(prompt, start, end)

Removed span: VKKPGASVKVS


In [ ]:
new_prompt

'EVQLVQSGAECKASGYTFTSYYMHWVRQAPGQGLEWMGIINPR'

In [ ]:
inputs = tokenizer(new_prompt, return_tensors="pt")
original_length = inputs["input_ids"].shape[1]
outputs = model.generate(inputs["input_ids"], max_length=original_length + 10, do_sample=True, top_k=2, top_p=0.95)


# extract only the generated tokens
generated_ids = outputs[0][original_length:]
predicted_span = tokenizer.decode(generated_ids, skip_special_tokens=True)

print("Predicted span:", predicted_span)

# insert predicted span back into original position
final_sequence = new_prompt[:start] + predicted_span + new_prompt[start:]

print("Final sequence:", final_sequence)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Predicted span: QGLEWQGLEWQG
Final sequence: EVQLVQSGAEQGLEWQGLEWQGCKASGYTFTSYYMHWVRQAPGQGLEWMGIINPR
